# Finally, we will evaluate the performance of Zero Shot Classification (used facebook/bart-large-mnli as the embedding model) from Part 2

In [1]:
import pandas as pd
from transformers import pipeline
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('final.csv')

In [5]:
data

,Unnamed: 0,ID,poem_author,language,original_poem,light_clean_poem,theme,keywords,keywords_ori,keywords_eng,keyword_eng_theme,key_sentences_ori,key_sentences_eng,poem_eng
0,0,1,Halyna Kruk,Ukrainian,"***\ncказати собі чесно, якомога чесніше,\nтак...","*** cказати собі чесно, якомога чесніше, так н...","{'maidan', 'protest', 'revolution', 'ukraine'}","[('покладаючись', 0.3382), ('патріотичних', 0....","['покладаючись', 'патріотичних', 'втікаючи', '...","['relying on', 'patriotic', 'running away', 'W...","[{'sequence': 'relying on', 'labels': ['maidan...","['я не хочу, щоб ми довіку ходили такими глухи...","[""I don't want us to walk down such dead ends ...","***\nto tell yourself honestly, as honestly as..."
1,1,2,Ostap Slyvynsky,Ukrainian,ХАЙКУ З МАЙДАНУ\nДим у рукавах.\nНіби йдеш з ц...,ХАЙКУ З МАЙДАНУ Дим у рукавах. Ніби йдеш з цим...,"{'haiku', 'protest', 'revolution', 'poetic for...","[('пообіцяти', 0.4741), ('пустимо', 0.3054), (...","['пообіцяти', 'пустимо', 'вистоїмо', 'лісистим...","['to promise', ""let's go"", ""let's stand"", 'woo...","[{'sequence': 'to promise', 'labels': ['langua...","['пустимо качу – хай їм', 'ми вистоїмо.', 'поо...","[""let's let the duck go - let them eat"", 'we w...",Haika from the Maidan\nSmoke in the sleeves.\n...
2,2,3,Osip Mandelstam,Russian,(О.М.)\nНе мучнистой бабочкою белой\nВ землю я...,(О.М.) Не мучнистой бабочкою белой В землю я з...,"{'time', 'war', 'twentieth century'}","[('восклицанья', 0.381), ('мыслящее', 0.3004),...","['восклицанья', 'мыслящее', 'молчаливо', 'позв...","['exclamations', 'thinking', 'silently', 'vert...","[{'sequence': 'exclamations', 'labels': ['lang...","['Я хочу, чтоб мыслящее тело', 'Пронесла пехот...","['I want a thinking body', 'The infantry carri...",Not powdery butterfly white\nI will return the...
3,3,4,Boris Khersonsky,Russian,"***\n""Союз нерушимый"" сыграют и хором споют.\n...","*** ""Союз нерушимый"" сыграют и хором споют. В ...","{'soviet union', 'anthem', 'morphine', 'food'}","[('проснусь', 0.3366), ('нерушимый', 0.331), (...","['проснусь', 'нерушимый', 'скрывать', 'подсыха...","['wake up', 'indestructible', 'hide', 'dries u...","[{'sequence': 'wake up', 'labels': ['time', 'p...",['Неужели им всем есть о чем подумать и что ск...,['Do they all have something to think about an...,"***\nThe ""indestructible union"" will play and ..."
4,4,5,Oleksandr Irvanets,Ukrainian,"***\nЦей рік, який почався з середи - \nКуди в...","*** Цей рік, який почався з середи - Куди він...","{'time', 'new year', 'seasons'}","[('наступний', 0.4395), ('четверга', 0.2653), ...","['наступний', 'четверга', 'кочерга', 'почнетьс...","['following', 'Thursday', 'poker', 'will begin...","[{'sequence': 'following', 'labels': ['family'...","['Чи буде він прямим, як кочерга? ', 'Але наст...","['Will it be straight as a poker?', 'But next ...",***\nThis year that started from Wednesday -\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,1005,1191,Halyna Kruk,Ukrainian,"#довоєнні_вірші як все змінилося, здається, що...","як все змінилося, здається, що безповоротно *...","{'school', 'tram', 'war', 'summer'}","[('повивітрюється', 0.4717), ('настовбурчиться...","['повивітрюється', 'настовбурчиться', 'наїжачи...","['airs out', 'grumble', 'get crowded', 'we bak...","[{'sequence': 'airs out', 'labels': ['time', '...","['повивітрюється, стерпне', 'скоро спекаємось ...","['airy, tolerable', 'soon we will be swelterin...","As everything has changed, it seems that irrev..."
1006,1006,1192,Halyna Kruk,Ukrainian,"#довоєнні_вірші\nіз циклу ""Інші""\nнайгірше в й...","із циклу ""Інші"" найгірше в його житті почина...","{'relationships', 'family', 'mortality', 'trav...","[('зрозумілої', 0.437), ('незнайомих', 0.2801)...","['зрозумілої', 'незнайомих', 'незнайомій', 'по...","['understandable', 'strangers', 'unfamiliar', ...","[{'sequence': 'understandable', 'labels': ['la...","['не знати, як повернутися додому ', 'висохлих...","['not knowing how to get home', 'dry, inflamed...","from t

#### Columns of interest: theme and keyword_eng_theme

## For each poem, we have multiple keywords. Each keyword have 3 labels (themes) in which their probabilities assigned by the Classifier are the highest 3.

In [4]:
data['keyword_eng_theme'][0]

"[{'sequence': 'relying on', 'labels': ['maidan', 'family', 'nation'], 'scores': [0.12305205315351486, 0.10531776398420334, 0.0666816458106041]}, {'sequence': 'patriotic', 'labels': ['nation', 'maidan', 'language'], 'scores': [0.4591561555862427, 0.1214393675327301, 0.08682616800069809]}, {'sequence': 'running away', 'labels': ['protest', 'time', 'animal'], 'scores': [0.1631479114294052, 0.15938355028629303, 0.12018370628356934]}, {'sequence': 'Want', 'labels': ['protest', 'maidan', 'language'], 'scores': [0.1297539621591568, 0.11201462894678116, 0.08314479887485504]}, {'sequence': 'small town', 'labels': ['maidan', 'life', 'geography'], 'scores': [0.2456987351179123, 0.15827718377113342, 0.15817879140377045]}]"

## Our task here is to assemble the themes, i.e. all the labels shown above, and evaluate how close those themes are to the human-annotated themes

### The evaluation metric we use here is the "majority vote". For every possible theme in the assembled list, what are the few most occurring theme?

In [11]:
theme = data['theme']

# Putting all labels together
predicted_theme = data['keyword_eng_theme'].apply(lambda x: [i['labels'] for i in eval(x)])

In [12]:
predicted_theme

0       [[maidan, family, nation], [nation, maidan, la...
1       [[language, maidan, time], [time, life, langua...
2       [[language, protest, time], [time, maidan, fam...
3       [[time, protest, life], [maidan, nature, life]...
4       [[family, time, language], [time, season, life...
                              ...                        
1005    [[time, protest, death], [protest, language, m...
1006    [[language, maidan, life], [maidan, protest, l...
1007    [[gender, family, language], [protest, maidan,...
1008    [[time, maidan, family], [animal, language, de...
1009    [[maidan, time, language], [maidan, family, na...
Name: keyword_eng_theme, Length: 1010, dtype: object

In [14]:
# Flattening the list, making a list in list --> list
predicted_theme = predicted_theme.apply(lambda x: [i for g in x for i in g])
predicted_theme

0       [maidan, family, nation, nation, maidan, langu...
1       [language, maidan, time, time, life, language,...
2       [language, protest, time, time, maidan, family...
3       [time, protest, life, maidan, nature, life, ti...
4       [family, time, language, time, season, life, g...
                              ...                        
1005    [time, protest, death, protest, language, maid...
1006    [language, maidan, life, maidan, protest, lang...
1007    [gender, family, language, protest, maidan, ti...
1008    [time, maidan, family, animal, language, death...
1009    [maidan, time, language, maidan, family, natio...
Name: keyword_eng_theme, Length: 1010, dtype: object

Great! Now we attempt to calculate the proportion. Below is a small example on the first row

In [23]:
pd.Series(predicted_theme[0]).value_counts(normalize=True)

maidan       0.266667
nation       0.133333
language     0.133333
protest      0.133333
family       0.066667
time         0.066667
animal       0.066667
life         0.066667
geography    0.066667
dtype: float64

In [26]:
# We only need the ordered index
list(pd.Series(predicted_theme[0]).value_counts(normalize=True).index)

['maidan',
 'nation',
 'language',
 'protest',
 'family',
 'time',
 'animal',
 'life',
 'geography']

In [27]:
def rearrange_themes(l):
    return list(pd.Series(l).value_counts(normalize=True).index)

In [29]:
predicted_theme = predicted_theme.apply(rearrange_themes)

/tmp/ipykernel_11234/3184082607.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return list(pd.Series(l).value_counts(normalize=True).index)


In [30]:
predicted_theme

0       [maidan, nation, language, protest, family, ti...
1       [language, maidan, time, nature, life, protest...
2       [time, family, maidan, life, language, protest...
3       [time, maidan, life, protest, nature, family, ...
4       [time, language, family, life, maidan, season,...
                              ...                        
1005    [time, maidan, protest, life, death, language,...
1006    [maidan, language, protest, time, family, life...
1007    [family, maidan, time, gender, language, prote...
1008      [language, time, maidan, family, animal, death]
1009     [maidan, language, time, family, nation, gender]
Name: keyword_eng_theme, Length: 1010, dtype: object

### Great! Let's compare in the following order:
1. Top 1 theme is in the human-annotated themes list
2. Top 2 themes are in the human-annotated themes list
3. Top 3 themes are in the human-annotated themes list

In [37]:
# Below is the human-annotated theme we've gotten from the "theme" column
theme

0          {'maidan', 'protest', 'revolution', 'ukraine'}
1       {'haiku', 'protest', 'revolution', 'poetic for...
2                    {'time', 'war', 'twentieth century'}
3          {'soviet union', 'anthem', 'morphine', 'food'}
4                         {'time', 'new year', 'seasons'}
                              ...                        
1005                  {'school', 'tram', 'war', 'summer'}
1006    {'relationships', 'family', 'mortality', 'trav...
1007    {'pre-war', 'fruit', 'nature', 'summer', 'seas...
1008            {'confession', 'apostle', 'christianity'}
1009              {'needle', 'speech', 'silence', 'pain'}
Name: theme, Length: 1010, dtype: object

In [39]:
eval(theme[0])

{'maidan', 'protest', 'revolution', 'ukraine'}

### Caveat: The human-annotated themes does not really follow exactly the list of big_themes we have set. Therefore, the results might or might not be below expectation. 

In [34]:
# Top 1 theme is in the human-annotated themes list
top1_predicted_theme = predicted_theme.apply(lambda x: x[0] if len(x) > 0 else None)
top1_predicted_theme

0         maidan
1       language
2           time
3           time
4           time
          ...   
1005        time
1006      maidan
1007      family
1008    language
1009      maidan
Name: keyword_eng_theme, Length: 1010, dtype: object

In [35]:
len(top1_predicted_theme)

1010

In [41]:
contains = 0
for i in range(len(top1_predicted_theme)):
    if top1_predicted_theme[i] in eval(theme[i]):
        contains += 1

In [43]:
contains/len(top1_predicted_theme)

0.05643564356435644

### 5% of the human-curated theme list contains the exact same theme as the top1 predicted theme

#### This is very much below expectation, but as mentioned, there is a high possibility for this to happen given the noise in the human-annotated themes (annotated by different individuals) and the unexplained uncertainty in the Classifier itself

In [46]:
# Top 2 themes are in the human-annotated themes list
top2_predicted_theme = predicted_theme.apply(lambda x: set(x[:2]) if len(x) > 0 else None)
top2_predicted_theme

0         {maidan, nation}
1       {maidan, language}
2           {time, family}
3           {maidan, time}
4         {time, language}
               ...        
1005        {maidan, time}
1006    {maidan, language}
1007      {maidan, family}
1008      {time, language}
1009    {maidan, language}
Name: keyword_eng_theme, Length: 1010, dtype: object

In [51]:
{'maidan', 'nation'}.intersection({'maidan', 'protest', 'revolution', 'ukraine'})

{'maidan'}

In [54]:
contains = 0
for i in range(len(top2_predicted_theme)):
    if top2_predicted_theme[i]:
        if top2_predicted_theme[i].intersection(eval(theme[i])):
            contains += 1

In [55]:
contains

137

In [56]:
contains/len(top1_predicted_theme)

0.13564356435643565

### Increased to 13.5% of the human-curated theme list contains the exact same theme as the top2 predicted themes

In [57]:
# Top 3 themes are in the human-annotated themes list
top3_predicted_theme = predicted_theme.apply(lambda x: set(x[:3]) if len(x) > 0 else None)
top3_predicted_theme

0        {maidan, nation, language}
1          {maidan, time, language}
2            {maidan, time, family}
3              {maidan, time, life}
4          {time, family, language}
                   ...             
1005        {maidan, time, protest}
1006    {maidan, protest, language}
1007         {time, maidan, family}
1008       {maidan, time, language}
1009       {maidan, time, language}
Name: keyword_eng_theme, Length: 1010, dtype: object

In [58]:
contains = 0
for i in range(len(top3_predicted_theme)):
    if top3_predicted_theme[i]:
        if top3_predicted_theme[i].intersection(eval(theme[i])):
            contains += 1

In [59]:
contains

190

In [60]:
contains/len(top1_predicted_theme)

0.18811881188118812

### Increased to 18.8% of the human-curated theme list contains the exact same theme as the top3 predicted themes!

Some key takeaways:
1. In this project, zero shot classification seems to work pretty good at associating words/short phrases (keyword) with another word (theme) given that there isn't much context fed into the model. The strength of zero shot classification is very obvious when compared to spacy's word similarity and WordNet from NLTK (we saw this in Part 1).

2. As for KeyBERT, this project did not explicitly evaluate how well KeyBERT is at extracting keywords. Generally, based on a few observations, we could see that KeyBERT also extracted metaphors "by accident" (well, not actually by accident, but this is more like a by-project). This is an area that I would continue to explore and validate in Spring as a passion project.

3. Another important consideration would be the explanability/interpretability of the models we have built. Here, we are putting a certain extent of trust in the backend mechanism of the models and commenting on the model solely based on its output. Not to forget that explanability and interpretability are important scopes in ethical AI.

Further improvement:
1. Considering the fact that the structure of poems is very different from typical passages/stories - there are more implicit meanings and less direct pointers to living/non-living things/incidents, it would be better if we could train a model on poem specifically instead of using a (general) pre-trained model.

2. Testing on different embedding models like the xlm series.

3. Notice that there is a huge variance in the human-annotated themes, it is key to ensure consistent annotation ofr better performance.

## Thank you!